In [1]:
import matplotlib.pyplot as plt
import pandas as pd 
import spotipy 
import librosa
import librosa.display
import numpy as np
import matplotlib as mpl
import urllib.request
import seaborn as sns
import tqdm
import scipy
import joblib
from pathlib import Path
from IPython.display import Audio, Markdown, Image
from spotipy.oauth2 import SpotifyClientCredentials 


# Números y Datos
import numpy as np
import pandas as pd
from scipy.spatial.distance import pdist, squareform
from scipy.interpolate import interp1d

# Análisis de sonido
import librosa
import librosa.display
import spotipy 
from spotipy.oauth2 import SpotifyClientCredentials 

from scipy.signal import savgol_filter

# Machine learning
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

sns.set_context('poster')
sns.set_style('darkgrid')

plt.rcParams['figure.figsize'] = [12, 6]
plt.rcParams['figure.dpi'] = 100 # 200 e.g. is really fine, but slower

## Levantar el dataset AA

In [2]:
#df_af = pd.read_pickle('audio_features.pickle')
#df_tracks = pd.read_pickle('tracks.pickle')
df_aa = pd.read_pickle('audio_analysis.pickle') \
    .set_index(['id','start']) \
    .sort_values(by=['id', 'start'], ascending=True, na_position='first')

## Desglozar las listas de pitches y timbre

In [6]:
df_m2 = df_aa

pitches_nombre = ['p00_C','p01_C#','p02_D','p03_D#','p04_E','p05_F',
                 'p06_F#','p07_G','p08_G#','p09_A','p10_A#','p11_B']
df1 = pd.DataFrame(df_m2['pitches'].tolist(), columns=pitches_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

timbre_nombre = ['t00','t01','t02','t03','t04','t05',
                 't06','t07','t08','t09','t10','t11']
df1 = pd.DataFrame(df_m2['timbre'].tolist(), columns=timbre_nombre, index=df_m2.index)
df_m2 = pd.concat([df_m2,df1],axis=1)

df_m2 = df_m2.drop(['pitches','timbre'],1)

df1 = None
df_m2.head(3).T

duration  confidence  loudness_start  \
id                     start                                           
00At7PWydsvg7g5xgaYan9 0.00000   0.24381         0.0         -60.000   
                       0.24381   0.17492         1.0         -60.000   
                       0.41873   0.17329         1.0         -34.279   

                                loudness_max_time  loudness_max  loudness_end  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000            0.00000       -60.000           0.0   
                       0.24381            0.01436       -12.457           0.0   
                       0.41873            0.01554       -12.728           0.0   

                                                                          pitches  \
id                     start                                                        
00At7PWydsvg7g5xgaYan9 0.00000  [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, ...   
                       0.24381  [0.121, 0.996, 0.08, 0.104, 1.0, 0.223, 0.043,...   
                       0.41873  [0.032, 1.0, 0.06, 0.059, 0.368, 0.084, 0.036,...   

                                                                           timbre  \
id                     start                                                        
00At7PWydsvg7g5xgaYan9 0.00000  [0.0, 171.13, 9.469, -28.48, 57.491, -50.067, ...   
                       0.24381  [37.641, -77.643, -30.613, 137.462, 36.96, -51...   
                       0.41873  [38.171, -17.741, -22.798, 149.223, 31.833, -5...   

                                p00_C  p01_C#  ...     t02      t03     t04  \
id                     start                   ...                            
00At7PWydsvg7g5xgaYan9 0.00000  1.000   1.000  ...   9.469  -28.480  57.491   
                       0.24381  0.121   0.996  ... -30.613  137.462  36.960   
                       0.41873  0.032   1.000  ... -22.798  149.223  31.833   

                                   t05     t06     t07     t08    t09     t10  \
id                     start                                                    
00At7PWydsvg7g5xgaYan9 0.00000 -50.067  14.833   5.359 -27.228  0.973 -10.640   
                       0.24381 -51.024 -18.745 -28.822  -7.170 -1.994 -15.695   
                       0.41873 -53.446 -36.485 -39.479 -25.795 -1.903 -21.908   

                                   t11  
id                     start            
00At7PWydsvg7g5xgaYan9 0.00000  -7.228  
                       0.24381   9.411  
                       0.41873  15.934  

[3 rows x 32 columns]

## Obtener los cuartiles para sintetizar la info más importante

In [15]:
df_m2_25 = df_m2.groupby('id').quantile(.25)
df_m2_50 = df_m2.groupby('id').quantile(.5)
df_m2_75 = df_m2.groupby('id').quantile(.75)

In [17]:
df_m2_75.head(3).T

id,00At7PWydsvg7g5xgaYan9,00BgcUYkIImsDN5rNjkR1U,00WgnFIsY1y84p7mYAJRuF
duration,0.272995,0.33900,0.269863
confidence,0.695000,0.71100,0.702000
loudness_start,-5.628500,-5.74900,-11.602000
loudness_max_time,0.057285,0.03646,0.035515
loudness_max,-0.174000,-0.42700,-3.615750
loudness_end,0.000000,0.00000,0.000000
p00_C,0.507500,0.39100,0.884250
p01_C#,1.000000,1.00000,0.585250
p02_D,0.405000,0.20600,0.631000
p03_D#,0.378000,0.19800,0.915250


In [18]:
df_m2_25.to_pickle("./audio_analysis_25.pickle")
df_m2_50.to_pickle("./audio_analysis_50.pickle")
df_m2_75.to_pickle("./audio_analysis_75.pickle")

## Levantar las otras fuentes de datos

In [41]:
df_af = pd.read_pickle('audio_features.pickle')
df_tracks = pd.read_pickle('tracks.pickle')
df_af = df_af.drop(['duration_ms','type','uri','track_href','analysis_url'],1)
df_tracks = df_tracks.drop(['uri','type','preview_url','href','external_urls','external_ids'],1)
df_tracks.head(3).T

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
album,"{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external..."
artists,Tommy Flanagan,Art Farmer,Christian Scott aTunde Adjuah
available_markets,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...","[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...","[CA, US]"
disc_number,1,1,1
duration_ms,349600,344066,348746
explicit,False,False,False
is_local,False,False,False
name,Angel Eyes - Studio,Fair Weather - Remastered,Cease Fire
popularity,49,47,13
track_number,3,2,5


## Mergear los datasets 
(inner join para maximizar la certeza de nuestro análisis)

In [49]:
df_m = df_tracks.merge(df_af, on='id', how='inner')
df_m = df_m.merge(df_m2_25.add_suffix('_25'), on='id', how='inner')
df_m = df_m.merge(df_m2_50.add_suffix('_50'), on='id', how='inner')
df_m = df_m.merge(df_m2_75.add_suffix('_75'), on='id', how='inner')
df_m.head(3).T

id,4d3XHYFFuYYzxWr2cJ6yQl,3X3p3u03P8eFL8WTH0oaaU,4KlXeaheot0OI9PoOWspvZ
album,"{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external...","{'album_type': 'ALBUM', 'artists': [{'external..."
artists,Tommy Flanagan,Art Farmer,Christian Scott aTunde Adjuah
available_markets,"[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...","[AD, AE, AL, AR, AT, AU, BA, BE, BG, BH, BO, B...","[CA, US]"
disc_number,1,1,1
duration_ms,349600,344066,348746
...,...,...,...
t07_75,12.614,21.081,10.6685
t08_75,8.861,-1.19175,14.4615
t09_75,15.507,10.2332,8.3895
t10_75,-1.126,-1.91975,2.203


In [53]:
df_m.to_pickle("./dataset_integrado.pickle")